In [1]:
# full_recode_with_7_datasets.py
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from copy import deepcopy
from sklearn.datasets import load_iris, load_wine, fetch_covtype
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# =======================================================================
# UTILITY FUNCTIONS
# =======================================================================

def set_npseed(seed):
    np.random.seed(seed)

def set_torchseed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def safe_read_csv(url, **kwargs):
    """Read CSV with a helpful error message."""
    try:
        return pd.read_csv(url, **kwargs)
    except Exception as e:
        raise RuntimeError(f"Failed to read CSV from {url}: {e}")

# =======================================================================
# DATA LOADING: 7 datasets
# - covertype (sklearn fetch)
# - poker (UCI)
# - higgs (UCI, huge -> sampled)
# - letter (UCI)
# - pendigits (UCI)
# - gas (UCI sensor drift)
# - satimage (UCI)
# Each loader returns (X, y, feature_names)
# =======================================================================

def load_dataset(dataset_name, sample_frac=0.05, random_state=42):
    """
    Load a dataset by name.
    - sample_frac: fraction to sample for very large datasets (0 < sample_frac <= 1.0)
    """
    dataset_name = dataset_name.lower()
    rng = np.random.RandomState(random_state)

    if dataset_name == 'iris':
        data = load_iris()
        X, y = data.data, data.target
        feature_names = data.feature_names

    elif dataset_name == 'wine':
        data = load_wine()
        X, y = data.data, data.target
        feature_names = data.feature_names

    elif dataset_name == 'covertype':
        # Large but manageable with sklearn
        try:
            print("[INFO] Fetching Covertype (may take a bit)...")
            cov = fetch_covtype()
            X = cov.data
            y = cov.target - 1  # make 0-indexed
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            if sample_frac < 1.0:
                idx = rng.choice(len(X), size=int(len(X)*sample_frac), replace=False)
                X = X[idx]
                y = y[idx]
        except Exception as e:
            raise RuntimeError(f"Error fetching covertype: {e}")

    elif dataset_name == 'poker':
        # Poker-hand (UCI) - very large ~1M rows
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand.data"
        try:
            print("[INFO] Downloading Poker dataset (UCI).")
            df = safe_read_csv(url, header=None)
            X = df.iloc[:, :-1].values
            y = df.iloc[:, -1].values
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            # sample if needed
            if sample_frac < 1.0:
                idx = rng.choice(len(X), size=int(len(X)*sample_frac), replace=False)
                X = X[idx]; y = y[idx]
        except Exception as e:
            raise RuntimeError(f"Error loading Poker dataset: {e}")

    elif dataset_name == 'higgs':
        # HIGGS dataset (very large). default: sample small fraction
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz"
        try:
            print("[INFO] Loading HIGGS (streaming, sampled). This may take time if sample_frac ~1.0")
            # read in chunks and sample rows to avoid loading full file into memory
            chunks = []
            chunk_iter = pd.read_csv(url, header=None, compression='gzip', chunksize=200000)
            for chunk in chunk_iter:
                if sample_frac >= 1.0:
                    chunks.append(chunk)
                else:
                    # sample rows from chunk
                    s = chunk.sample(frac=sample_frac, random_state=random_state)
                    chunks.append(s)
            df = pd.concat(chunks, ignore_index=True)
            # First column is label in original HIGGS (0/1); rest are features
            y = df.iloc[:, 0].values
            X = df.iloc[:, 1:].values
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            # convert to multiclass placeholder if you want multi-class - here keep binary as-is
        except Exception as e:
            raise RuntimeError(f"Error loading HIGGS dataset: {e}")

    elif dataset_name == 'letter':
        # Letter Recognition
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data"
        try:
            df = safe_read_csv(url, header=None)
            # first column is letter A-Z
            labels = df.iloc[:, 0].values
            unique = np.unique(labels)
            label_map = {ch: i for i, ch in enumerate(unique)}
            y = np.array([label_map[ch] for ch in labels])
            X = df.iloc[:, 1:].values
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            if sample_frac < 1.0:
                idx = rng.choice(len(X), size=int(len(X)*sample_frac), replace=False)
                X = X[idx]; y = y[idx]
        except Exception as e:
            raise RuntimeError(f"Error loading Letter dataset: {e}")

    elif dataset_name == 'pendigits' or dataset_name == 'pen':
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pendigits/pendigits.tra"
        try:
            df = safe_read_csv(url, header=None)
            X = df.iloc[:, :-1].values
            y = df.iloc[:, -1].values
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            if sample_frac < 1.0:
                idx = rng.choice(len(X), size=int(len(X)*sample_frac), replace=False)
                X = X[idx]; y = y[idx]
        except Exception as e:
            raise RuntimeError(f"Error loading PenDigits dataset: {e}")

    elif dataset_name == 'gas':
        # Gas Sensor Array Drift dataset (a cleaned / packaged CSV location may vary)
        # Using the UCI page: actual download is more complex; try a packaged CSV if available
        # We'll try an archived CSV (if reachable) else raise informative error
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00336/GasSensors.csv"
        try:
            df = safe_read_csv(url, header=0)
            # This dataset formats vary - try to detect last column as label if integer-like
            if df.shape[1] < 2:
                raise RuntimeError("Unexpected gas dataset format.")
            X = df.iloc[:, :-1].values
            y_raw = df.iloc[:, -1].values
            # If labels are strings, map to ints
            unique = np.unique(y_raw)
            if not np.issubdtype(y_raw.dtype, np.integer):
                map_ = {v: i for i, v in enumerate(unique)}
                y = np.array([map_[v] for v in y_raw])
            else:
                y = y_raw
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            if sample_frac < 1.0:
                idx = rng.choice(len(X), size=int(len(X)*sample_frac), replace=False)
                X = X[idx]; y = y[idx]
        except Exception as e:
            # As a safer fallback: try the "gas sensor array drift dataset" page files
            raise RuntimeError(f"Error loading Gas dataset: {e}")

    elif dataset_name == 'satimage':
        try:
            train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/satimage/sat.trn"
            test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/satimage/sat.tst"
            df_train = safe_read_csv(train_url, header=None, sep=r'\s+')
            df_test = safe_read_csv(test_url, header=None, sep=r'\s+')
            df = pd.concat([df_train, df_test], ignore_index=True)
            X = df.iloc[:, :-1].values
            y = df.iloc[:, -1].values - 1  # 0-index
            feature_names = [f'feature_{i}' for i in range(X.shape[1])]
        except Exception as e:
            raise RuntimeError(f"Error loading Satimage dataset: {e}")

    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    # Ensure numeric arrays
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int64)
    return X, y, feature_names

# =======================================================================
# DLGN MODEL (kept close to your original, small cleanup)
# =======================================================================

class DLGN_FC(nn.Module):
    def __init__(self, input_dim=None, output_dim=None, num_hidden_nodes=None, 
                 beta=30, dlgn_mode='dlgn_sf', mode='pwc', num_classes=4):
        super(DLGN_FC, self).__init__()
        if num_hidden_nodes is None:
            num_hidden_nodes = []
        self.num_hidden_layers = len(num_hidden_nodes)
        self.beta = beta
        self.dlgn_mode = dlgn_mode
        self.mode = mode
        self.num_classes = num_classes
        self.num_nodes = [input_dim] + list(num_hidden_nodes) + [num_classes]
        self.gating_layers = nn.ModuleList()
        self.value_layers = nn.ModuleList()

        for i in range(self.num_hidden_layers+1):
            if i != self.num_hidden_layers:
                if self.dlgn_mode == 'dlgn_sf':
                    temp = nn.Linear(self.num_nodes[0], self.num_nodes[i+1], bias=False)
                else:
                    temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
                self.gating_layers.append(temp)
            temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
            self.value_layers.append(temp)
    
    def return_gating_functions(self):
        effective_weights = []
        for i in range(self.num_hidden_layers):
            curr_weight = self.gating_layers[i].weight.detach().clone()
            if self.dlgn_mode == 'dlgn_sf':
                effective_weights.append(curr_weight)
            else:
                if i == 0:
                    effective_weights.append(curr_weight)
                else:
                    effective_weights.append(torch.matmul(curr_weight, effective_weights[-1]))
        return effective_weights

    def forward(self, x):
        # determine device once
        device = x.device if x.is_cuda else torch.device('cpu')
        gate_scores = [x.to(device)]

        if self.mode == 'pwc':
            values = [torch.ones_like(x).to(device)]
        else:
            values = [x.to(device)]

        for i in range(self.num_hidden_layers):
            if self.dlgn_mode == 'dlgn_sf':
                gate_scores.append((x.to(device) @ self.gating_layers[i].weight.T))
            else:
                gate_scores.append(self.gating_layers[i](gate_scores[-1]))
            curr_gate_on_off = torch.sigmoid(self.beta * gate_scores[-1])
            values.append(self.value_layers[i](values[-1]) * curr_gate_on_off)
        
        values.append(self.value_layers[self.num_hidden_layers](values[-1]))
        return values, gate_scores

# =======================================================================
# TRAINING FUNCTION (FULL DATA)
# =======================================================================

def train_dlgn_full(DLGN_obj, data_full, labels_full, 
                    num_epoch=1500, parameter_mask=None, seed=42, lr=0.001, 
                    no_of_batches=10, saved_epochs=None, x_epoch=1000):
    if parameter_mask is None:
        parameter_mask = {name: torch.ones_like(param) for name, param in DLGN_obj.named_parameters()}
    if saved_epochs is None:
        saved_epochs = list(range(0, num_epoch, 100)) + [num_epoch-1]

    set_torchseed(seed)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    DLGN_obj.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(DLGN_obj.parameters(), lr=lr)

    data_torch = torch.Tensor(data_full)
    labels_torch = torch.tensor(labels_full, dtype=torch.int64)

    batch_size = max(1, len(data_full) // max(1, no_of_batches))
    losses = []
    DLGN_obj_store = []
    train_losses = []

    for epoch in tqdm(range(saved_epochs[-1] + 1), desc="Training on Full Data"):
        if epoch in saved_epochs:
            DLGN_obj_copy = deepcopy(DLGN_obj)
            DLGN_obj_copy.to(torch.device('cpu'))
            DLGN_obj_store.append(DLGN_obj_copy)

            train_outputs_values, _ = DLGN_obj(torch.Tensor(data_full).to(device))
            targets = torch.tensor(labels_full, dtype=torch.int64).to(device)
            train_loss = criterion(train_outputs_values[-1], targets)
            train_losses.append(train_loss.cpu().item())

            if train_loss.item() < 5e-6 or np.isnan(train_loss.detach().cpu().numpy()):
                break

        running_loss = 0.0
        for batch_start in range(0, len(data_full), batch_size):
            if (batch_start + batch_size) > len(data_full):
                # make last batch smaller instead of skipping
                inputs = data_torch[batch_start:len(data_full)]
                targets = labels_torch[batch_start:len(data_full)]
            else:
                inputs = data_torch[batch_start:batch_start+batch_size]
                targets = labels_torch[batch_start:batch_start+batch_size]
            
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            values, _ = DLGN_obj(inputs)
            outputs = values[-1]
            loss = criterion(outputs, targets)
            loss.backward()

            for name, param in DLGN_obj.named_parameters():
                parameter_mask[name] = parameter_mask[name].to(device)
                if param.grad is not None:
                    param.grad *= parameter_mask[name]
                    if "gat" in name and epoch > x_epoch:
                        param.grad *= 0.

            optimizer.step()
            running_loss += loss.item()

        # Track loss after epoch
        train_outputs_values, _ = DLGN_obj(torch.Tensor(data_full).to(device))
        targets = torch.tensor(labels_full, dtype=torch.int64).to(device)
        train_loss = criterion(train_outputs_values[-1], targets)
        losses.append(train_loss.cpu().detach().clone().numpy())

    DLGN_obj.to(torch.device('cpu'))
    return train_losses, DLGN_obj, DLGN_obj_store, losses

# =======================================================================
# BASELINE MODELS (FULL DATA)
# =======================================================================

def train_baselines_full(X_full, y_full):
    results = {}
    print("  Training Random Forest on full data...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20)
    rf.fit(X_full, y_full)
    rf_pred = rf.predict(X_full)
    results['Random Forest'] = accuracy_score(y_full, rf_pred)

    print("  Training SVM on full data (may be slow for large datasets)...")
    svm = SVC(kernel='rbf', random_state=42)
    svm.fit(X_full, y_full)
    results['SVM'] = accuracy_score(y_full, svm.predict(X_full))

    print("  Training CART on full data...")
    cart = DecisionTreeClassifier(random_state=42, max_depth=20)
    cart.fit(X_full, y_full)
    results['CART'] = accuracy_score(y_full, cart.predict(X_full))

    return results

# =======================================================================
# MAIN EXPERIMENT RUNNER (FULL DATASET)
# =======================================================================

def run_experiment_full(dataset_name, config, sample_frac=None):
    """
    dataset_name: str
    config: config dict for DLGN training
    sample_frac: override the default sampling fraction (None uses loader default)
    """
    print("\n" + "="*80)
    print(f"EXPERIMENT: {dataset_name.upper()} Dataset (FULL DATA)")
    print("="*80)

    # Load data
    print("\n1. Loading data...")
    try:
        if sample_frac is None:
            X, y, feature_names = load_dataset(dataset_name)
        else:
            X, y, feature_names = load_dataset(dataset_name, sample_frac=sample_frac)
    except Exception as e:
        print(f"[ERROR] Could not load dataset {dataset_name}: {e}")
        return None

    num_classes = len(np.unique(y))
    input_dim = X.shape[1]

    print(f"   Total Samples: {X.shape[0]}")
    print(f"   Features: {input_dim}")
    print(f"   Classes: {num_classes}")

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    print(f"   Using ALL {len(X_scaled)} samples for training AND testing")

    # Train baselines
    print("\n2. Training Baseline Models on Full Data...")
    baseline_results = train_baselines_full(X_scaled, y)

    # Train DLGN
    print("\n3. Training DLGN on Full Data...")

    set_torchseed(config.get('seed', 6675))
    DLGN_model = DLGN_FC(
        input_dim=input_dim,
        output_dim=1,
        num_hidden_nodes=config['hidden_nodes'],
        beta=config['beta'],
        dlgn_mode=config.get('dlgn_mode', 'dlgn'),
        mode='pwc',
        num_classes=num_classes
    )

    train_parameter_masks = {name: torch.ones_like(param) for name, param in DLGN_model.named_parameters()}

    train_losses, DLGN_final, DLGN_store, losses = train_dlgn_full(
        DLGN_obj=deepcopy(DLGN_model),
        data_full=X_scaled,
        labels_full=y,
        parameter_mask=train_parameter_masks,
        lr=config['lr'],
        no_of_batches=config['batches'],
        saved_epochs=config['saved_epochs'],
        x_epoch=config['x_epoch'],
        num_epoch=config['num_epochs'],
        seed=config.get('seed', 5000)
    )

    # Evaluate DLGN on full data
    full_outputs_values, _ = DLGN_final(torch.Tensor(X_scaled))
    full_logits = full_outputs_values[-1].detach().numpy()
    full_preds = np.argmax(full_logits, axis=1)
    dlgn_acc = accuracy_score(y, full_preds)

    # Print results
    print("\n" + "="*80)
    print("RESULTS (Training and Testing on FULL DATASET)")
    print("="*80)
    print(f"\n{'Model':<20} {'Accuracy on Full Data':<25}")
    print("-"*45)
    print(f"{'DLGN':<20} {dlgn_acc:.4f}")
    for model_name, acc in baseline_results.items():
        print(f"{model_name:<20} {acc:.4f}")

    # Per-class accuracy for DLGN
    print(f"\nDLGN Per-Class Accuracy (on full data):")
    for i in range(num_classes):
        class_mask = y == i
        class_correct = np.sum((y[class_mask] == full_preds[class_mask]))
        class_total = np.sum(class_mask)
        if class_total > 0:
            print(f"  Class {i}: {class_correct / class_total:.4f} ({class_correct}/{class_total})")

    return {
        'dataset': dataset_name,
        'dlgn_acc': dlgn_acc,
        'baseline_results': baseline_results,
        'num_samples': X.shape[0],
        'num_features': input_dim,
        'num_classes': num_classes
    }

# =======================================================================
# RUNNING MULTIPLE EXPERIMENTS
# =======================================================================

if __name__ == "__main__":
    # DLGN configs tuned by dataset scale
    small_dataset_config = {
        'hidden_nodes': [10, 10, 10],
        'beta': 4,
        'lr': 0.002,
        'batches': 5,
        'dlgn_mode': 'dlgn',
        'num_epochs': 500,
        'saved_epochs': list(range(0, 501, 100)),
        'x_epoch': 300,
        'seed': 6675
    }

    medium_dataset_config = {
        'hidden_nodes': [20, 20, 20],
        'beta': 4,
        'lr': 0.002,
        'batches': 10,
        'dlgn_mode': 'dlgn',
        'num_epochs': 800,
        'saved_epochs': list(range(0, 801, 100)),
        'x_epoch': 500,
        'seed': 6675
    }

    large_dataset_config = {
        'hidden_nodes': [64, 64, 32],
        'beta': 4,
        'lr': 0.001,
        'batches': 50,
        'dlgn_mode': 'dlgn',
        'num_epochs': 400,
        'saved_epochs': list(range(0, 401, 50)),
        'x_epoch': 300,
        'seed': 6675
    }

    # Datasets to run (the 7 requested)
    datasets_and_configs = [
        ('covertype', large_dataset_config, 0.02),  # sample 2% by default to speed up testing
        ('poker', large_dataset_config, 0.01),      # sample 1% (1M rows -> ~10k rows)
        ('higgs', large_dataset_config, 0.001),     # sample 0.1% by default (11M -> ~11k rows)
        ('letter', medium_dataset_config, 1.0),
        ('pendigits', medium_dataset_config, 1.0),
        ('gas', medium_dataset_config, 1.0),
        ('satimage', medium_dataset_config, 1.0)
    ]

    all_results = []
    for name, cfg, sample_frac in datasets_and_configs:
        try:
            res = run_experiment_full(name, cfg, sample_frac=sample_frac)
            if res:
                all_results.append(res)
        except Exception as e:
            print(f"[ERROR] Experiment {name} failed: {e}")
            continue

    # Summary
    if all_results:
        print("\n" + "="*80)
        print("SUMMARY OF ALL EXPERIMENTS (FULL DATASET RESULTS)")
        print("="*80)
        print(f"\n{'Dataset':<15} {'Samples':<10} {'Features':<10} {'Classes':<10} {'DLGN':<10} {'RF':<10} {'SVM':<10} {'CART':<10}")
        print("-"*95)
        for r in all_results:
            ds = r['dataset']
            print(f"{ds:<15} {r['num_samples']:<10} {r['num_features']:<10} {r['num_classes']:<10} "
                  f"{r['dlgn_acc']:.4f}     {r['baseline_results']['Random Forest']:.4f}     "
                  f"{r['baseline_results']['SVM']:.4f}     {r['baseline_results']['CART']:.4f}")
        print("\nALL EXPERIMENTS COMPLETE!")
    else:
        print("No successful experiments completed.")



EXPERIMENT: COVERTYPE Dataset (FULL DATA)

1. Loading data...
[INFO] Fetching Covertype (may take a bit)...
   Total Samples: 11620
   Features: 54
   Classes: 7
   Using ALL 11620 samples for training AND testing

2. Training Baseline Models on Full Data...
  Training Random Forest on full data...
  Training SVM on full data (may be slow for large datasets)...
  Training CART on full data...

3. Training DLGN on Full Data...


Training on Full Data: 100%|██████████| 401/401 [01:02<00:00,  6.42it/s]



RESULTS (Training and Testing on FULL DATASET)

Model                Accuracy on Full Data    
---------------------------------------------
DLGN                 0.9897
Random Forest        0.9661
SVM                  0.7550
CART                 0.9801

DLGN Per-Class Accuracy (on full data):
  Class 0: 0.9928 (4271/4302)
  Class 1: 0.9852 (5516/5599)
  Class 2: 1.0000 (728/728)
  Class 3: 1.0000 (52/52)
  Class 4: 1.0000 (193/193)
  Class 5: 0.9828 (343/349)
  Class 6: 1.0000 (397/397)

EXPERIMENT: POKER Dataset (FULL DATA)

1. Loading data...
[INFO] Downloading Poker dataset (UCI).
[ERROR] Could not load dataset poker: Error loading Poker dataset: Failed to read CSV from https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand.data: HTTP Error 404: Not Found

EXPERIMENT: HIGGS Dataset (FULL DATA)

1. Loading data...
[INFO] Loading HIGGS (streaming, sampled). This may take time if sample_frac ~1.0
   Total Samples: 11000
   Features: 28
   Classes: 2
   Using ALL 11

Training on Full Data: 100%|██████████| 401/401 [00:57<00:00,  6.98it/s]



RESULTS (Training and Testing on FULL DATASET)

Model                Accuracy on Full Data    
---------------------------------------------
DLGN                 1.0000
Random Forest        0.9999
SVM                  0.7616
CART                 0.9885

DLGN Per-Class Accuracy (on full data):
  Class 0: 1.0000 (5112/5112)
  Class 1: 1.0000 (5888/5888)

EXPERIMENT: LETTER Dataset (FULL DATA)

1. Loading data...
   Total Samples: 20000
   Features: 16
   Classes: 26
   Using ALL 20000 samples for training AND testing

2. Training Baseline Models on Full Data...
  Training Random Forest on full data...
  Training SVM on full data (may be slow for large datasets)...
  Training CART on full data...

3. Training DLGN on Full Data...


...


RESULTS (Training and Testing on FULL DATASET)

Model                Accuracy on Full Data    
---------------------------------------------
DLGN                 0.9851
Random Forest        0.9998
SVM                  0.9627
CART                 0.9861

DLGN Per-Class Accuracy (on full data):
  Class 0: 1.0000 (789/789)
  Class 1: 0.9569 (733/766)
  Class 2: 0.9932 (731/736)
  Class 3: 0.9789 (788/805)
  Class 4: 0.9792 (752/768)
  Class 5: 0.9897 (767/775)
  Class 6: 0.9780 (756/773)
  Class 7: 0.9700 (712/734)
  Class 8: 0.9669 (730/755)
  Class 9: 0.9772 (730/747)
  Class 10: 0.9878 (730/739)
  Class 11: 0.9895 (753/761)
  Class 12: 0.9937 (787/792)
  Class 13: 0.9860 (772/783)
  Class 14: 0.9867 (743/753)
  Class 15: 0.9714 (780/803)
  Class 16: 1.0000 (783/783)
  Class 17: 0.9644 (731/758)
  Class 18: 0.9973 (746/748)
  Class 19: 0.9937 (791/796)
  Class 20: 0.9938 (808/813)
  Class 21: 0.9751 (745/764)
  Class 22: 0.9973 (750/752)
  Class 23: 0.9975 (785/787)
  Class 24: 0.9898 

Training on Full Data: 100%|██████████| 801/801 [00:24<00:00, 32.11it/s]



RESULTS (Training and Testing on FULL DATASET)

Model                Accuracy on Full Data    
---------------------------------------------
DLGN                 1.0000
Random Forest        1.0000
SVM                  0.9976
CART                 1.0000

DLGN Per-Class Accuracy (on full data):
  Class 0: 1.0000 (780/780)
  Class 1: 1.0000 (779/779)
  Class 2: 1.0000 (780/780)
  Class 3: 1.0000 (719/719)
  Class 4: 1.0000 (780/780)
  Class 5: 1.0000 (720/720)
  Class 6: 1.0000 (720/720)
  Class 7: 1.0000 (778/778)
  Class 8: 1.0000 (719/719)
  Class 9: 1.0000 (719/719)

EXPERIMENT: GAS Dataset (FULL DATA)

1. Loading data...
[ERROR] Could not load dataset gas: Error loading Gas dataset: Failed to read CSV from https://archive.ics.uci.edu/ml/machine-learning-databases/00336/GasSensors.csv: HTTP Error 404: Not Found

EXPERIMENT: SATIMAGE Dataset (FULL DATA)

1. Loading data...
   Total Samples: 6435
   Features: 36
   Classes: 6
   Using ALL 6435 samples for training AND testing

2. Train

Training on Full Data:   0%|          | 0/801 [00:00<?, ?it/s]

[ERROR] Experiment satimage failed: Target 6 is out of bounds.

SUMMARY OF ALL EXPERIMENTS (FULL DATASET RESULTS)

Dataset         Samples    Features   Classes    DLGN       RF         SVM        CART      
-----------------------------------------------------------------------------------------------
covertype       11620      54         7          0.9897     0.9661     0.7550     0.9801
higgs           11000      28         2          1.0000     0.9999     0.7616     0.9885
letter          20000      16         26         0.9851     0.9998     0.9627     0.9861
pendigits       7494       16         10         1.0000     1.0000     0.9976     1.0000

ALL EXPERIMENTS COMPLETE!
